## KOSPI 리스트 기얻어오기


In [65]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas as pd
import warnings

# 모든 컬럼 출력
pd.set_option('display.max_columns', None)

# 모든 행 출력
pd.set_option('display.max_rows', None)

# 열 너비 무제한(문자열 길어도 줄바꿈 없이 출력)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore", category=FutureWarning)

dict_tickers = {}
dict_tickers_inv = {}
tickers = stock.get_market_ticker_list(market='KOSPI')
for ticker in tickers:
    ticker_nm = stock.get_market_ticker_name(ticker)
    dict_tickers_inv[ticker_nm] = ticker
    dict_tickers[ticker] = ticker_nm

In [66]:
# =========================================
# 1) 가장 최근 영업일 자동 계산
# =========================================
def get_last_business_day():
    today = datetime.today()
    # YYYYMMDD 문자열
    today_str = today.strftime("%Y%m%d")

    # 주말이거나 휴장이면 직전 영업일 찾기
    while True:
        try:
            # 시가총액 조회 시도(성공하면 영업일)
            stock.get_market_cap(today_str)
            return today_str
        except:
            # 하루 전으로 이동
            today -= timedelta(days=1)
            today_str = today.strftime("%Y%m%d")


dt = get_last_business_day()
print("사용 날짜:", dt)

사용 날짜: 20251118


In [171]:
import pandas as pd
import numpy as np
from pykrx import stock
from datetime import datetime


def get_financial_data(list_dt):
    # ================================================
    # 1. 오늘 날짜 설정
    # ================================================
    today = datetime.now().strftime("%Y%m%d")
    list_dt.append(today)

    # 결과 저장
    rows = []
    for dt in list_dt:
        # ================================================
        # 2. KOSPI 종목 전체
        # ================================================
        tickers = stock.get_market_ticker_list(dt, market="KOSPI")

        # ================================================
        # 3. Daily 데이터 로딩
        # ================================================
        fund = stock.get_market_fundamental(dt)  # EPS, PER, DIV, BPS 등
        price_df = stock.get_market_ohlcv(dt)  # 종가
        mcap_df = stock.get_market_cap(dt)           # 시가총액

        # 결과 저장
        # rows = []

        for ticker in tickers:
            try:
                name = stock.get_market_ticker_name(ticker)
                price = price_df.loc[ticker]["종가"] # 주가
                mcap = mcap_df.loc[ticker]["시가총액"]
                volume = mcap_df.loc[ticker]["거래량"]
                trading_value = mcap_df.loc[ticker]["거래대금"]
                #shares_outstanding = mcap_df.loc[ticker]["상장주식수"]
                bps = fund.loc[ticker]['BPS']  # 자기자본(=자산 - 부채) / 방행주식수
                per = fund.loc[ticker]['PER']
                pbr = fund.loc[ticker]['PBR']
                div = fund.loc[ticker]["DIV"]  # 배당수익률 %
                eps = fund.loc[ticker]["EPS"]
                dps = fund.loc[ticker]['DPS']  # 주당 배당금


                # ===============================
                # 기본 값
                # ===============================
                #shares = mcap / price  if price                   # 발행주식수
                if price in [0, None] or pd.isna(price) or mcap == 0 or pd.isna(mcap):
                    shares = np.nan
                else:
                    shares = mcap / price
                dps = price * (div / 100)                # 주당 배당금
                total_div = mcap * (div / 100)           # 총배당금
                net_income = eps * shares                # 연간 순이익(TTM)
                if per in [0, None] or pd.isna(per) or pbr == 0 or pd.isna(pbr):
                    roe = np.nan
                else:
                    roe = pbr / per

                # ===============================
                # 연간 총주주환원율
                # ===============================
                srr_year = total_div / net_income if net_income > 0 else None


                rows.append({
                    "dt": dt,
                    "yy": dt[:4],
                    "티커": ticker,
                    "종목명": name,
                    "주가": price,
                    "BPS": bps,
                    "PER": per,
                    'PBR': pbr,
                    'ROE': np.round(roe, 2),
                    'DIV': div,
                    'EPS': eps,
                    'DPS': dps,
                    '시가총액': mcap,
                    '거래량': volume,
                    '거래대금': trading_value,
                    # 연간 값
                    "순이익": round(net_income, 0),
                    "총배당금": round(total_div, 0),
                    "주식수":shares,
                    "주주환원율": round(srr_year, 4) if srr_year else None,

                })
            except:
                continue

    # ================================================
    # 4. DataFrame으로 변환
    # ================================================
    df = pd.DataFrame(rows)
    return df

In [199]:
import pandas as pd
from kr_holidays import is_working_day
from datetime import datetime, timedelta

def get_trading_days(start_date: str, end_date: str):
    """
    start_date, end_date는 'YYYYMMDD' 문자열
    한국 주식시장 영업일(주말 + 공휴일 제외) 리스트 반환
    """
    start = datetime.strptime(start_date, "%Y%m%d")
    end = datetime.strptime(end_date, "%Y%m%d")

    date_list = []
    current = start
    while current <= end:
        s = current.strftime("%Y%m%d")
        # 주말 아니고 kr_holidays로 공휴일 확인
        if current.weekday() < 5 and is_working_day(s) :
            date_list.append(s)
        current += timedelta(days=1)

    return date_list


# =========================
# 사용 예시
# =========================
list_dt = []
list_year = [2020, 2021, 2022, 2023, 2024]
for year in list_year:
    calendar_dt = get_trading_days(f"{year}0101", f"{year}1231")
    list_dt.append(calendar_dt[-2])

df = get_financial_data(list_dt)


In [200]:
df.head()

,dt,yy,티커,종목명,주가,BPS,PER,PBR,ROE,DIV,EPS,DPS,시가총액,거래량,거래대금,순이익,총배당금,주식수,주주환원율
0,20201230,2020,095570,AJ네트웍스,4615.0,6802.0,4.70,0.68,0.14,6.50,982.0,299.9750,216084891425,120954,560421850,4.597949e+10,1.404552e+10,46822295.0,0.3055
1,20201230,2020,006840,AK홀딩스,25150.0,62448.0,11.60,0.40,0.03,2.98,2168.0,749.4700,333176159150,89368,2252587900,2.872071e+10,9.928650e+09,13247561.0,0.3457
2,20201230,2020,027410,BGF,4895.0,15699.0,17.42,0.31,0.02,2.25,281.0,110.1375,468533691945,469185,2291176695,2.689642e+10,1.054201e+10,95716791.0,0.3919
3,20201230,2020,282330,BGF리테일,135500.0,36022.0,15.46,3.76,0.24,1.99,8763.0,2696.4500,2341969263000,25983,3485099000,1.514589e+11,4.660519e+10,17283906.0,0.3077
4,20201230,2020,138930,BNK금융지주,5680.0,25415.0,3.45,0.22,0.06,6.34,1647.0,360.1120,1851312197280,1947256,10955767300,5.368154e+11,1.173732e+11,325935246.0,0.2186


(981.9999211913897, 4.69959266802444, 0.6784769185533667)

In [201]:
# 주주환원율이 있어야 하며 PER 10 이하 이면서 시가총액이 상위 25%인 회사
df_filter = df[(df.yy == '2025') & (df['주주환원율'] > 0) & (df['PER'] < 10) & (df['시가총액'] >= df['시가총액'].describe()['75%'])].copy()
list_filtered_tickers = df_filter['티커'].unique().tolist()
len(df_filter)

64

In [224]:
dict_ticker_results = {
                        'ticker':[], '종목명':[], '주가':[], 'EPS':[], 'PER':[],
                        '순이익_slope':[], '총배당금_slope':[], '주주환원율_slope':[],
                        '순이익':[], '총배당금':[], '주주환원율':[]
                       }
for ticker in list_filtered_tickers:
    df_tmp = df[df['티커'] == ticker].copy()
    df_tmp = df_tmp.sort_values(by='yy')

    for col in ['순이익', '총배당금', '주주환원율']:
        y = np.array(df_tmp[col].values)
        if np.any((y == 0) | pd.isna(y)):
            slope = np.nan
            dict_ticker_results[f'{col}'].append(False)
        else:
            y_pct = y / y[0]
            x = np.arange(len(y))
            slope = np.polyfit(x, y_pct, 1)[0]
            dict_ticker_results[f'{col}'].append((y_pct[1:] > 1).all())
        dict_ticker_results[f'{col}_slope'].append(slope)



    df_tmp = df_tmp[df_tmp.yy == '2025'].copy()
    dict_ticker_results['ticker'].append(ticker)
    dict_ticker_results['종목명'].append(df_tmp['종목명'].values[0])
    dict_ticker_results['EPS'].append(df_tmp['EPS'].values[0])
    dict_ticker_results['PER'].append(df_tmp['PER'].values[0])
    dict_ticker_results['주가'].append(df_tmp['주가'].values[0])

In [234]:
df_result = pd.DataFrame(dict_ticker_results)
df_result_filterd = df_result[(df_result['순이익']==True) & (df_result['총배당금']==True) ].copy()
df_result_filterd.sort_values(by=['순이익_slope', '총배당금_slope'], ascending=[False, False])

,ticker,종목명,주가,EPS,PER,순이익_slope,총배당금_slope,주주환원율_slope,순이익,총배당금,주주환원율
3,005830,DB손해보험,122100.0,30838.0,3.96,0.888861,0.703044,-0.032343,True,True,False
21,009450,경동나비엔,58400.0,8603.0,6.79,0.526726,0.251745,-0.059167,True,True,False
34,000810,삼성화재,464000.0,48779.0,9.51,0.453353,0.245697,-0.062303,True,True,False
17,001120,LX인터내셔널,30250.0,4885.0,6.19,0.436536,1.109755,0.067366,True,True,False
58,300720,한일시멘트,17340.0,2657.0,6.53,0.299193,0.216149,-0.019710,True,True,False
42,271560,오리온,104900.0,13271.0,7.90,0.265959,0.538410,0.114731,True,True,False
13,175330,JB금융지주,23400.0,3439.0,6.80,0.213887,0.474117,0.128384,True,True,True
33,029780,삼성카드,53000.0,6228.0,8.51,0.191398,0.148416,-0.021841,True,True,False
54,000240,한국앤컴퍼니,25650.0,3703.0,6.93,0.184986,0.328190,0.084610,True,True,True
41,007310,오뚜기,384500.0,40048.0,9.60,0.168556,0.069619,-0.047800,True,True,False


In [235]:
df[df['종목명'] == 'DB손해보험']

,dt,yy,티커,종목명,주가,BPS,PER,PBR,ROE,DIV,EPS,DPS,시가총액,거래량,거래대금,순이익,총배당금,주식수,주주환원율
19,20201230,2020,005830,DB손해보험,43750.0,84026.0,7.33,0.52,0.07,3.43,5966.0,1500.625,3097500000000,142810,6244023750,4.223928e+11,1.062442e+11,70800000.0,0.2515
914,20211230,2021,005830,DB손해보험,54000.0,90596.0,5.89,0.60,0.10,4.07,9171.0,2197.800,3823200000000,148954,8057207800,6.493068e+11,1.556042e+11,70800000.0,0.2396
1829,20221229,2022,005830,DB손해보험,65300.0,110605.0,4.51,0.59,0.13,5.36,14482.0,3500.080,4623240000000,164431,10729375600,1.025326e+12,2.478057e+11,70800000.0,0.2417
2744,20231228,2023,005830,DB손해보험,83700.0,89508.0,5.11,0.94,0.18,5.50,16373.0,4603.500,5925960000000,116101,9666318600,1.159208e+12,3.259278e+11,70800000.0,0.2812
3668,20241230,2024,005830,DB손해보험,102800.0,170486.0,3.55,0.60,0.17,5.16,28955.0,5304.480,7278240000000,81056,8402558225,2.050014e+12,3.755572e+11,70800000.0,0.1832
4597,20251118,2025,005830,DB손해보험,122100.0,154832.0,3.96,0.79,0.20,5.57,30838.0,6800.970,8644680000000,313234,38459787614,2.183330e+12,4.815087e+11,70800000.0,0.2205
